In [22]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder

In [23]:
url='https://drive.google.com/file/d/1N7z7Nn4IuQjZSv1eqLh32HKrKKaLrsoQ/view?usp=sharing'
file_id=url.split('/')[-2]
dwn_url='https://drive.google.com/uc?id=' + file_id
df=pd.read_csv(dwn_url)
df = df.rename({'Risk':'labels'}, axis = 'columns')
df

,Prompt,Sex,labels
0,"Using the following characteristics, assess wh...",male,good
1,"Using the following characteristics, assess wh...",female,bad
2,"Using the following characteristics, assess wh...",male,good
3,"Using the following characteristics, assess wh...",male,good
4,"Using the following characteristics, assess wh...",male,bad
...,...,...,...
995,"Using the following characteristics, assess wh...",female,good
996,"Using the following characteristics, assess wh...",male,good
997,"Using the following characteristics, assess wh...",male,good
998,"Using the following characteristics, assess wh...",male,bad


In [24]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [25]:
# Tokenize responses 
tokenized_data = tokenizer(
    df['Prompt'].tolist(),
    max_length = 512,
    padding = True,
    truncation = True,
    return_tensors="pt"
)

# Encode labels
le = LabelEncoder()
labels = le.fit_transform(df['labels']) 
labels = torch.tensor(labels.tolist())

In [26]:
from torch.utils.data import Dataset

class SentimentData(Dataset):
    def __init__(self, tokenized_data, labels):
        self.input_ids = tokenized_data['input_ids']
        self.attention_mask = tokenized_data['attention_mask']
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

dataset = SentimentData(tokenized_data, labels)

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

num_param = count_parameters(model.pre_classifier) + count_parameters(model.classifier)
print("Parameters in last 2 layers:", num_param)

Parameters in last 2 layers: 592899


In [29]:
for param in model.parameters():
    param.requires_grad = False

In [30]:
def make_adapter(in_dim, bottleneck_dim, out_dim):
    adapter_layers = torch.nn.Sequential(
        torch.nn.Linear(in_dim, bottleneck_dim),
        torch.nn.GELU(),
        torch.nn.Linear(bottleneck_dim, out_dim),
    )
    return adapter_layers

In [31]:
total_size = 0
bottleneck_size = 32 # hyperparameter

for block_idx in range(6):

    ###################################################
    # insert 1st adapter layer into transformer block
    ###################################################

    orig_layer_1 = model.distilbert.transformer.layer[block_idx].attention.out_lin

    adapter_layers_1 = make_adapter(
        in_dim=orig_layer_1.out_features, 
        bottleneck_dim=bottleneck_size, 
        out_dim=orig_layer_1.out_features)

    new_1 = torch.nn.Sequential(orig_layer_1, *adapter_layers_1)
    model.distilbert.transformer.layer[block_idx].attention.out_lin = new_1
    
    total_size += count_parameters(adapter_layers_1)

    ###################################################
    # insert 2nd adapter layer into transformer block
    ###################################################

    orig_layer_2 = model.distilbert.transformer.layer[block_idx].ffn.lin2

    adapter_layers_2 = make_adapter(
        in_dim=orig_layer_2.out_features, 
        bottleneck_dim=bottleneck_size, 
        out_dim=orig_layer_2.out_features)

    new_2 = torch.nn.Sequential(orig_layer_2, *adapter_layers_2)
    model.distilbert.transformer.layer[block_idx].ffn.lin2 = new_2
    
    total_size += count_parameters(adapter_layers_2)
    

print("Number of adapter parameters added:", total_size)

Number of adapter parameters added: 599424


In [32]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset)
eval_dataloader = DataLoader(dataset)

In [33]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr = 5e-5)

In [34]:
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs*len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Sequential(
              (0): Linear(in_features=768, out_features=768, bias=True)
              (1): Linear(in_features=768, out_features=32, bias=True)
              (2): GELU(approximate='none')
              (3): Line

In [35]:
from tqdm.auto import tqdm
model.train()

progress_bar = tqdm(range(num_training_steps))

# Training loop
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], labels=batch['labels'])
        
        # Compute loss
        loss = outputs.loss
        loss.backward() 

        # Update parameters
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        # Update progress bar
        progress_bar.update(1)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    model.save_pretrained("./fine_tuned_distilbert_CS")

  0%|          | 0/3000 [00:00<?, ?it/s]

Epoch 1/3, Loss: 0.3685868978500366
Epoch 2/3, Loss: 0.4493759870529175
Epoch 3/3, Loss: 0.3839338421821594


In [36]:
import evaluate

metric = evaluate.load("accuracy")

model.eval()

for batch in eval_dataloader:

    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():

        outputs = model(**batch)

    logits = outputs.logits

    predictions = torch.argmax(logits, dim=-1)

    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.7}

In [37]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("./fine_tuned_distilbert_CS")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at ./fine_tuned_distilbert_CS were not used when initializing DistilBertForSequenceClassification: ['distilbert.transformer.layer.0.attention.out_lin.0.bias', 'distilbert.transformer.layer.0.attention.out_lin.0.weight', 'distilbert.transformer.layer.0.attention.out_lin.1.bias', 'distilbert.transformer.layer.0.attention.out_lin.1.weight', 'distilbert.transformer.layer.0.attention.out_lin.3.bias', 'distilbert.transformer.layer.0.attention.out_lin.3.weight', 'distilbert.transformer.layer.0.ffn.lin2.0.bias', 'distilbert.transformer.layer.0.ffn.lin2.0.weight', 'distilbert.transformer.layer.0.ffn.lin2.1.bias', 'distilbert.transformer.layer.0.ffn.lin2.1.weight', 'distilbert.transformer.layer.0.ffn.lin2.3.bias', 'distilbert.transformer.layer.0.ffn.lin2.3.weight', 'distilbert.transformer.layer.1.attention.out_lin.0.bias', 'distilbert.transformer.layer.1.attention.out_lin.0.weight', 'distilbert.transformer.layer.1.attention.out_lin.1.bias', 'distilbert.transf

In [38]:
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [39]:
def get_prediction(text):
    # Tokenize the input
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)

    predicted_class = torch.argmax(probabilities, dim=1).item()
    
    label_mapping = {1: 'bad', 2: 'good'} 
    predicted_label = label_mapping[predicted_class]

    return predicted_label

# Example instance to predict
input_text = df['Prompt'][5]
predicted_label = get_prediction(input_text)
print(predicted_label)

good


In [40]:
predictions = []
for index, row in df.iterrows():
    input_text = row['Prompt'] 
    predicted_label = get_prediction(input_text) 
    predictions.append(predicted_label)

df['Predicted_Label'] = predictions
df

,Prompt,Sex,labels,Predicted_Label
0,"Using the following characteristics, assess wh...",male,good,good
1,"Using the following characteristics, assess wh...",female,bad,good
2,"Using the following characteristics, assess wh...",male,good,good
3,"Using the following characteristics, assess wh...",male,good,good
4,"Using the following characteristics, assess wh...",male,bad,good
...,...,...,...,...
995,"Using the following characteristics, assess wh...",female,good,good
996,"Using the following characteristics, assess wh...",male,good,good
997,"Using the following characteristics, assess wh...",male,good,good
998,"Using the following characteristics, assess wh...",male,bad,good
